In [1]:
import torch 
import torch.nn as nn

In [2]:
class MLP(nn.Module):
    def __init__(self,input_dim, output_dim):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()

        self.linear_1 = nn.Linear(input_dim, 256)
        self.linear_2 = nn.Linear(256, 128)
        self.linear_3 = nn.Linear(128, 64)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.5)  
        self.output = nn.Linear(64, output_dim)
        
    def forward(self, x):
        x = self.flatten(x)
        x = self.relu(self.linear_1(x))
        x = self.dropout(x)
        x = self.relu(self.linear_2(x))
        x = self.dropout(x)
        x = self.relu(self.linear_3(x))
        return self.output(x)

In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

image_size = 64 * 14 * 14
trgt_cnt = 10

model = MLP(image_size, trgt_cnt).to(device)

model.load_state_dict(torch.load("mnist_model.pt"))




<All keys matched successfully>

In [4]:
from PIL import Image
import torchvision.transforms as transforms


img = Image.open("test_image.jpg").convert("L")  # Convert to grayscale

transforms = transforms.Compose([
    transforms.Resize((28, 28)),
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: 1.0 -x)  # Add batch dimension
])

img_tensor = transforms(img).unsqueeze(0).to(device)
print(img_tensor.shape)

torch.Size([1, 1, 28, 28])


In [5]:
model.eval()
with torch.no_grad():
    output = model(img_tensor)
    pred_target = torch.argmax(output, dim = 1)
    print(pred_target.item())

RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x784 and 12544x256)